In [1]:
from icrl import *
from model import *
import pickle

d:\Anaconda3\envs\textgen\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


bin d:\Anaconda3\envs\textgen\lib\site-packages\bitsandbytes\libbitsandbytes_cuda118.dll


In [2]:
# load trajectory pkl
traj = pickle.load(open('data/linucb_trajectories.pkl', 'rb'))

In [9]:
import wandb

wandb.login(key = 'da9f2bbc59300d7434138182a83a9b35c0cea793') # login to wandb
wandb.init(project="icrl_pretrain", reinit=True)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


test_loss,█▇▅▃▂▂▂▁▁▁▁▁▁▁▁
train_loss,█▆▅▃▂▂▁▁▁▁▁▁▁▁▁
test_loss,566.0982
train_loss,556.83414


In [10]:
# split trajectory into train and test
import tqdm
from sklearn.model_selection import train_test_split

train_traj, test_traj = train_test_split(traj, test_size=0.2)

batch_size = 8

# create dataset
train_dataset = TrajectoryDataset(train_traj, 10)
test_dataset = TrajectoryDataset(test_traj, 10)

# create dataloader
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# set up model
# model = TransformerModel(embed_dim=70, num_heads=5)
config = {
        'horizon': 200,
        'dim':5,
        'act_num':10,
        'state_dim': 50,
        'action_dim': 10,
        'n_layer': 8,
        'n_embd': 32,
        'n_head': 4,
        'shuffle': True,
        'dropout': 0.2,
        'test': False,
        'act_type': 'relu',

    }
action_dim = config['action_dim']
model = Transformer(config)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Using device {device}")
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=5e-4, weight_decay=1e-4)
loss_fn = nn.CrossEntropyLoss(reduction='sum')
# Trainer
def trainer(model,train_dataloader, test_dataloader, optimizer, num_epochs=10):
    # DataLoader can handle batching and shuffling
    model.train()  # Set the model to training mode

    for epoch in range(num_epochs):
        print(f"Epoch {epoch}")
        epoch_loss = 0.0
        for batch in tqdm.tqdm(train_dataloader):
            # Unpack the data
            # tokens, action_labels = batch
            # print(batch['context_actions'].size())
            pred_actions = model(batch) # dimension: (batch_size, num_actions, seq_len)
            # print(pred_actions.shape)
            true_actions = batch['true_actions'] # dimension: (batch_size, seq_len)
            # true_actions = true_actions.unsqueeze(-1).repeat(1, 1, action_dim) # dimension: (batch_size, seq_len, num_actions)
            # print(true_actions)
            # print(pred_actions)
            # tokens  = tokens.to(device)
            # action_labels = action_labels.to(device)
            pred_actions_flat =  pred_actions.view(-1, action_dim)
            true_actions_flat = true_actions.view(-1)
            loss = loss_fn(pred_actions_flat, true_actions_flat)
            # loss.backward()
            # print(action_labels)
            # Reset the gradients in the optimizer
            optimizer.zero_grad()
            # # type
            # # Forward pass
            # logits = model(tokens)  # Your forward method should handle the causal mask internally
            
            # # Compute loss
            # loss = loss(logits, action_labels)
    
            # # Backward pass
            loss.backward()
            
            # Update parameters
            optimizer.step()
            
            epoch_loss += loss.item()
        epoch_loss /= len(train_dataloader)
        # Compute the test loss
        test_loss = 0.0
        model.eval()
        with torch.no_grad():
            for batch in tqdm.tqdm(test_dataloader):
                pred_actions = model(batch)
                true_actions = batch['true_actions']
                pred_actions_flat =  pred_actions.view(-1, action_dim)
                true_actions_flat = true_actions.view(-1)
                loss = loss_fn(pred_actions_flat, true_actions_flat)
                
                test_loss += loss.item()
        model.train()
        test_loss /= len(test_dataloader)
        # wandb logging
        wandb.log({"train_loss": epoch_loss, "test_loss": test_loss})
        print(f"Epoch {epoch} | Train Loss {epoch_loss} | Test Loss {test_loss}")


Using device cuda


In [11]:
trainer(model, train_dataloader, test_dataloader, optimizer, num_epochs=15)

Epoch 0


100%|██████████| 250/250 [00:05<00:00, 48.53it/s]


Epoch 0 | Train Loss 3240.8949246826173 | Test Loss 2563.71881640625
Epoch 1


100%|██████████| 250/250 [00:04<00:00, 51.14it/s]


Epoch 1 | Train Loss 2171.1034239501955 | Test Loss 1762.7617829589844
Epoch 2


...

Epoch 12 | Train Loss 542.0763882446289 | Test Loss 588.7243215332031
Epoch 13


100%|██████████| 250/250 [00:05<00:00, 48.39it/s]


Epoch 13 | Train Loss 554.4992309875488 | Test Loss 544.5778474121093
Epoch 14


100%|██████████| 250/250 [00:04<00:00, 50.20it/s]

Epoch 14 | Train Loss 529.9145792541503 | Test Loss 538.7370557861328


In [12]:
# save model
torch.save(model.state_dict(), 'model/transformer_model.pth')

In [7]:
# evaluate model
